# MNIST convolutional neural networks with batch norm and regularizer

* MNIST data를 가지고 softmax classifier를 만들어보자.
* train data, validation data를 구분하여 model을 training하자
  + train data : model을 training 시키는 데 사용
  + validation data : training에 이용하지않은 data, model을 validation 하는 데 활용 (보통 1 epoch마다)
* batch normalization과 l2 regularizer를 추가!
* [`tf.contrib.slim`](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/contrib/slim) 참고
* [05_mnist_cnn_with_train_validation_split.ipynb](https://nbviewer.jupyter.org/github/modulabs/modu-tensorflow/blob/master/week03/05_mnist_cnn_with_train_validation_split.ipynb)를 기반으로 refactoring

### Import modules

In [ ]:
"""A LeNet-5 like cnn MNIST classifier.
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

slim = tf.contrib.slim

sess_config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))

#np.random.seed(219)
#tf.set_random_seed(219)

### Import data

In [ ]:
# Load training and test data from tf.keras
(x_train, y_train), (x_test, y_test) = \
    tf.keras.datasets.mnist.load_data()

x_train = x_train / 255.
x_train = x_train.astype(dtype = np.float32)
y_train = np.asarray(y_train, dtype=np.int32)

x_test = x_test / 255.
x_test = x_test.astype(np.float32)
y_test = np.asarray(y_test, dtype=np.int32)

### Split data

In [ ]:
# x_train에서 training에 쓸 index 50000개 가져옴
tr_indices = np.random.choice(np.arange(x_train.shape[0]), size = 50000, replace = False)

# model training에 이용할 data
x_tr = x_train[tr_indices]
y_tr = y_train[tr_indices]

# epoch 마다 model validation에 이용할 data
x_val = np.delete(arr = x_train, obj = tr_indices, axis = 0)
y_val = np.delete(arr = y_train, obj = tr_indices, axis = 0)

print('for training {}, {}'.format(x_tr.shape, y_tr.shape))
print('for validation {}, {}'.format(x_val.shape, y_val.shape))

### Set up dataset with `tf.data`

#### create input pipeline with `tf.data.Dataset` to train and validation data

In [ ]:
batch_size = 32

# for training
tr_dataset = tf.data.Dataset.from_tensor_slices((x_tr, y_tr))
tr_dataset = tr_dataset.shuffle(buffer_size = 10000)
tr_dataset = tr_dataset.batch(batch_size = batch_size)
tr_iterator = tr_dataset.make_initializable_iterator()

print(tr_dataset)

# for validation
# validation data의 용량이 in memory에 넣을 수 없을정도로 아래와 같이 활용한다.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.shuffle(buffer_size = 10000)
val_dataset = val_dataset.batch(batch_size = batch_size)
val_iterator = val_dataset.make_initializable_iterator()

print(val_dataset)

#### Define Iterator

In [ ]:
# tf.data.Iterator.from_string_handle의 output_shapes는 default = None이지만 꼭 값을 넣는 게 좋음
handle = tf.placeholder(tf.string)
iterator = tf.data.Iterator.from_string_handle(handle,
                                               tr_dataset.output_types,
                                               tr_dataset.output_shapes)
x, y = iterator.get_next()

### Create the model

In [ ]:
def cnn_model_fn(x):
    """"Model function for CNN.
    Args:
        x: input images
        mode: boolean whether trainig mode or test mode
    Returns:
    logits: unnormalized score funtion
  """
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    # MNIST images are 28x28 pixels, and have one color channel

    # batch normalization 적용 여부를 control하기위한 placeholder 
    is_training = tf.placeholder(tf.bool)
    
    with tf.name_scope('reshape'):
        x_image = tf.reshape(x, [-1, 28, 28, 1])
    
    # slim.arg_scope를 이용하여, slim.conv2d, slim.fully_connected 함수의 default로 batch norm과
    # l2 regularizer를 설정 (scale = 0.0003)
    
    with slim.arg_scope([slim.conv2d, slim.fully_connected],
                         weights_regularizer = slim.l2_regularizer(scale = .0003),
                         normalizer_fn = slim.batch_norm,
                         normalizer_params = {'decay' : .9, 'is_training': is_training}):
        # Convolutional Layer #1
        # Input Tensor Shape: [batch_size, 28, 28, 1]
        # Output Tensor Shape: [batch_size, 28, 28, 32]
        conv1 = slim.conv2d(x_image, 32, [5, 5], scope='conv1')

        # Pooling Layer #1
        # Input Tensor Shape: [batch_size, 28, 28, 32]
        # Output Tensor Shape: [batch_size, 14, 14, 32]
        pool1 = slim.max_pool2d(conv1, [2, 2], scope='pool1')
  
        # Convolutional Layer #2
        # Input Tensor Shape: [batch_size, 14, 14, 32]
        # Output Tensor Shape: [batch_size, 14, 14, 64]
        conv2 = slim.conv2d(pool1, 64, [5, 5], scope='conv2')

        # Pooling Layer #2
        # Second max pooling layer with a 2x2 filter and stride of 2
        # Input Tensor Shape: [batch_size, 14, 14, 64]
        # Output Tensor Shape: [batch_size, 7, 7, 64]
        pool2 = slim.max_pool2d(conv2, [2, 2], scope='pool2')

        # Flatten tensor into a batch of vectors
        # Input Tensor Shape: [batch_size, 7, 7, 64]
        # Output Tensor Shape: [batch_size, 7 * 7 * 64]
        pool2_flat = slim.flatten(pool2, scope='flatten')
  
        # Fully connected Layer
        # Input Tensor Shape: [batch_size, 7 * 7 * 64]
        # Output Tensor Shape: [batch_size, 1024]
        fc1 = slim.fully_connected(pool2_flat, 1024, scope='fc1')

        # Logits layer
        # Input Tensor Shape: [batch_size, 1024]
        # Output Tensor Shape: [batch_size, 10]
        logits = slim.fully_connected(fc1, 10, activation_fn=None, normalizer_fn = None,
                                      normalizer_params = None,
                                      scope='logits')
  
    return logits, is_training, x_image

### Build the model

In [ ]:
logits, is_training, x_image = cnn_model_fn(x)

### Define loss and optimizer

In [ ]:
cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=logits)

# tf.get_collection과 tf.GraphKeys.REGULARIZATION_LOSSES를 이용하여, regularization loss를 가져옴
reg_term = tf.reduce_sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES))
total_loss = cross_entropy + reg_term

# loss를 계산하기전 training의 경우, mini-batch 당 mean과 variance를 계산해야하고, 
# validation의 경우 training시 mini-batch당 가지고 있던 mean과 variance를 exponential moving average를 이용하여
# average한 것을 가지고 있다가 이용해야함. 아래의 코드가 그 기능을 함
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(control_inputs = update_ops):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(total_loss)

### `tf.Session()` and train

In [ ]:
sess = tf.Session(config=sess_config)
sess.run(tf.global_variables_initializer())

# history
tr_hist = []
val_hist = []

# Generate handles of tr_iterator and val_iterator
tr_handle, val_handle = sess.run(fetches = [tr_iterator.string_handle(), val_iterator.string_handle()])

# Train
max_epochs = 5

for epoch in range(max_epochs):
    
    avg_tr_loss = 0
    avg_val_loss = 0
    tr_step = 0
    val_step = 0
    
    # training 1-epoch
    sess.run(tr_iterator.initializer)
    while True:
        try:
            _, tr_loss = sess.run(fetches = [train_step, total_loss],
                               feed_dict = {handle : tr_handle, is_training : True})
            tr_step += 1
            avg_tr_loss += tr_loss

        except tf.errors.OutOfRangeError:
            break
    
    # validation 1-epoch
    sess.run(val_iterator.initializer)
    while True:
        try:
            val_loss = sess.run(total_loss, 
                                feed_dict = {handle : val_handle, is_training : False})
            val_step += 1
            avg_val_loss += val_loss
        
        except tf.errors.OutOfRangeError:
            break
            
    avg_tr_loss /= tr_step
    avg_val_loss /= val_step
    tr_hist.append(avg_tr_loss)
    val_hist.append(avg_val_loss)
    
    print('epochs : {:3}, tr_loss : {:.3f}, val_loss : {:.3f}'.format(epoch + 1, avg_tr_loss, avg_val_loss))

### Test trained model

In [ ]:
yhat = sess.run(logits, feed_dict = {x : x_test, is_training : False})
yhat = np.argmax(yhat, axis = 1)

In [ ]:
print('test accuracy : {:.2%}'.format(np.mean(yhat == y_test)))

### Visualize history

In [ ]:
plt.plot(tr_hist, label = 'train')
plt.plot(val_hist, label = 'validation')
plt.legend()